In [1]:
import random
import numpy as np
import pandas as pd

import os
import sys
import argparse
import torch
import torch.nn.functional as F

import src.data.data as data
import src.data.config as cfg
import src.interactive.functions as interactive
import src.models.utils as model_utils

In [2]:
#Source - Forward -> <- Backward - Sink
source="Gruber admits that they attempt to steal $640 million"
sink="Karl is gunned down by Powell"

In [3]:
#Load Comet Model

#Model Args
device=0
#Model weights available at
#https://drive.google.com/open?id=1FccEsYPUHnjzmX-Y5vjCBeyRt1pLo8FB
model_file="/home/tslab/lab/comet_pretrained_models/atomic_pretrained_model.pickle"

In [4]:
#Load Model in interactive Mode
opt, state_dict = interactive.load_model_file(model_file)
data_loader, text_encoder = interactive.load_data("atomic", opt)
n_ctx = data_loader.max_event + data_loader.max_effect
n_vocab = len(text_encoder.encoder) + n_ctx
model = interactive.make_model(opt, n_vocab, n_ctx, state_dict)
model.eval()

Loading data from: data/atomic/processed/generation/categories_oEffect#oReact#oWant#xAttr#xEffect#xIntent#xNeed#xReact#xWant-maxe1_17-maxe2_35-maxr_1.pickle
52


LMModel(
  (transformer): TransformerModel(
    (embed): Embedding(40542, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): Block(
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_1): LayerNorm()
        (mlp): MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm()
      )
      (1): Block(
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_1): LayerNorm()
        (mlp): MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm()
      )
    

In [5]:
#Prefixes used in 'expand' function
prefixes = ['tries', 'starts', 'wants', 'begins', ]
random.seed(0)

In [6]:
#Functions taken from C2PO - https://github.com/rajammanabrolu/C2PO/blob/master/C2PO/scripts/text-gen/generate.py

#Query Input with Comet model
def query(input_event, category='xNeed', sampling_algorithm='beam-10 '):
    sampler = interactive.set_sampler(opt, sampling_algorithm, data_loader)

    outputs = interactive.get_atomic_sequence(input_event, model, sampler, data_loader, text_encoder, category)[category]
    for k, v in outputs.items():
        outputs[k] = v[::-1]
    return outputs

In [7]:
# Convert event inputs to torch tensors

def set_atomic_inputs(input_event, output_event, category, data_loader, text_encoder):
    in_prefix, _ = data.atomic_data.do_example(text_encoder, input_event, None, True, None)
    # out_prefix, _ = data.atomic_data.do_example(text_encoder, output_event, None, True, None)

    # XMB = torch.zeros(1, data_loader.max_event + 1 + len(out_prefix)).long().to(cfg.device)
    XMB = torch.zeros(1, data_loader.max_event + 1).long().to(cfg.device)

    XMB[:, :len(in_prefix)] = torch.LongTensor(in_prefix)
    XMB[:, data_loader.max_event] = torch.LongTensor([text_encoder.encoder["<{}>".format(category)]])
    # XMB[:, data_loader.max_event + 1:] = torch.LongTensor(out_prefix)

    batch = {}
    batch["sequences"] = XMB
    batch["attention_mask"] = data.atomic_data.make_attention_mask(XMB)

    return batch

In [8]:

#Branching: forward, backward
def expand(event,character, width=2, depth=2, direction='forward'):
    #Relation to be used with comet
    if direction == 'forward':
        category = 'xWant'
    elif direction == 'backward':
        category = 'xNeed'

    stack = [[event]]
    done = set()

    outputs = []

    while len(stack) > 0:
        curr = stack.pop(0)
        if len(curr) != 1:
            outputs.append(curr)

        # don't repeat previous actions
        # print(f"Curr: {curr[-1]}, Character:{character.split()}")
        previous_action = curr[-1][len(character.split()) + 1:]
        # print("Previous action:",previous_action)
        if previous_action in done:
            continue
        done.add(previous_action)

        # get predictions with comet model
        out = query(curr[-1][len(character.split()) + 1:], category=category)

        reqs = out['beams'][:width]
        reqs = [[character, random.choice(prefixes)] + r.split() for r in reqs]
        # print(f"reqs: {reqs}")
        
        # add new beams
        for r in reqs:
            if len(r) <= 3 or len(curr) >= depth or len(r) > 12:
                continue
            # print(f"Insert: ",curr + [" ".join(r)])
            stack.insert(0, curr + [" ".join(r)])

    return outputs


In [9]:
width=2
depth=2
#Backward Branching
endings = expand(sink,"Karl",width=width, depth=depth, direction='backward')

#Forward Branching
beginnings = expand(source,"Gruber",width=width, depth=depth, direction='forward')


Input Event:   rl is gunned down by Powell
Target Effect: xNeed

Candidate Sequences:
none
to get into a car
to have a gun
to get into an accident
to get in the car
to be in the wrong place
to be driving
to be in a car
to drive a car
get into a car


Input Event:   rl begins to drive a car
Target Effect: xNeed

Candidate Sequences:
start the car
to get in the car
to start the car
to get into the car
to have a car
to open the car door
to get the keys
to get a license
to have a license
sit in car


Input Event:   rl begins get into a car
Target Effect: xNeed

Candidate Sequences:
to open the car door
to get in the car
to open the door
to get into the car
to have a car
open the car door
to go to the car
to open the car door .
to open the car
to put on his shoes


Input Event:   uber admits that they attempt to steal $640 million
Target Effect: xWant

Candidate Sequences:
to go to the bank
to get the money back
to spend the money
to get the money
to buy a bigger house
to buy a new car
to w

In [10]:
#Calculate Path Probability
def getProb(input_event, output_event, category='xNeed'):
    batch = set_atomic_inputs(input_event, output_event, category, data_loader, text_encoder)

    start_idx = data_loader.max_event + data.atomic_data.num_delimiter_tokens["category"]

    XMB = batch["sequences"][:, :start_idx]
    MMB = batch["attention_mask"][:, :start_idx]

    XMB = model_utils.prepare_position_embeddings(opt, data_loader.vocab_encoder, XMB.unsqueeze(-1))

    beam_ll = 0
    for w in output_event.split():
        lm_probs = F.log_softmax(model(
            XMB.unsqueeze(1), sequence_mask=MMB), dim=-1)
        dist = lm_probs[:, -1, :].squeeze()

        word = w + '</w>'
        # import ipdb; ipdb.set_trace()
        if word not in data_loader.vocab_encoder:
            return -1000
        else:
            tok_ll = dist[data_loader.vocab_encoder[w + '</w>']]
            next_tok = torch.tensor([[data_loader.vocab_encoder[w + '</w>']]], dtype=torch.long, device=MMB.device)

        beam_ll += tok_ll
        next_pos = XMB[:, -1:, 1] + 1

        next_x = torch.cat((next_tok, next_pos), -1).unsqueeze(1)
        XMB = torch.cat((XMB, next_x), 1)
        MMB = torch.cat([MMB, torch.ones(XMB.size(0), 1, device=MMB.device)], 1)
    return beam_ll

In [11]:
all=[]
fronts=[]
backs=[]
#Subject character in source event
b_character="Gruber"
#Subject character in sink event
e_character="Karl"

#Check all path combinations
for b in beginnings:
    for e in endings:
        b_action = ' '.join(b[-1].split()[len(b_character.split()) + 1:])
        e_action = ' '.join(e[-1].split()[len(e_character.split()) + 1:])
        print(f"b:{b}\ne:{e}")
        print(f"b_action:{b_action}\ne_action:{e_action}")
        forward_prob = getProb(b_action, e_action, category='xWant')
        backward_prob = getProb(e_action, b_action, category='xNeed')

        # normalize by probability of predicting "to"
        forward_prob /= getProb(b[-1], 'to', category='xWant')
        backward_prob /= getProb(e[-1], 'to', category='xNeed')
        print(f"Fwd Prob:{forward_prob} Bwd Prob:{backward_prob}\n")
        all.append((forward_prob + backward_prob, b + e[::-1]))
        fronts.append(forward_prob.detach().cpu().numpy())
        backs.append(backward_prob.detach().cpu().numpy())

b:['Gruber admits that they attempt to steal $640 million', 'Gruber begins to get a job']
e:['Karl is gunned down by Powell', 'Karl begins to drive a car']
b_action:to get a job
e_action:to drive a car
Fwd Prob:20.392353057861328 Bwd Prob:12.578056335449219

b:['Gruber admits that they attempt to steal $640 million', 'Gruber begins to get a job']
e:['Karl is gunned down by Powell', 'Karl begins get into a car']
b_action:to get a job
e_action:get into a car
Fwd Prob:23.434112548828125 Bwd Prob:19.131315231323242

b:['Gruber admits that they attempt to steal $640 million', 'Gruber wants to buy something']
e:['Karl is gunned down by Powell', 'Karl begins to drive a car']
b_action:to buy something
e_action:to drive a car
Fwd Prob:24.16367530822754 Bwd Prob:13.487824440002441

b:['Gruber admits that they attempt to steal $640 million', 'Gruber wants to buy something']
e:['Karl is gunned down by Powell', 'Karl begins get into a car']
b_action:to buy something
e_action:get into a car
Fwd Prob